## Test everything with only lexica based approach

In [6]:
import pandas as pd
from textblob import TextBlob

In [3]:
# Preprocess text (username and link placeholders)
def preprocess(text):

    # emojis entfernen
    #text = ersetze_emojis(text)

    # lemmatizer = WordNetLemmatizer()
    # words = text.split()
    # lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # text = ' '.join(lemmatized_words)


    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [9]:
# Laden Sie Ihren CSV-Datensatz
csv_file = "../../data/val_data.csv"  # Ersetzen Sie durch den Pfad zu Ihrer CSV-Datei
df = pd.read_csv(csv_file)

# Leere Listen zum Speichern der vorhergesagten Sentiments und der tatsächlichen Labels
predicted_sentiments = []
actual_labels = []

count = 0
# Durchlaufen Sie jeden Eintrag im DataFrame und wenden Sie das Modell an
for index, row in df.iterrows():
    text = row['text']
    text = preprocess(text)
    blob = TextBlob(text)
    score = blob.sentiment.polarity
    if (score > 0.15):
        predicted_label_id = 2
        # Speichern Sie das vorhergesagte Sentiment und das tatsächliche Label
        predicted_sentiments.append(predicted_label_id)
        actual_labels.append(row['label'])
    elif (score < -0.15):
        predicted_label_id = 0
        # Speichern Sie das vorhergesagte Sentiment und das tatsächliche Label
        predicted_sentiments.append(predicted_label_id)
        actual_labels.append(row['label'])
    else:
        predicted_label_id = 1
        # Speichern Sie das vorhergesagte Sentiment und das tatsächliche Label
        predicted_sentiments.append(predicted_label_id)
        actual_labels.append(row['label'])
    

# Fügen Sie die vorhergesagten Sentiments und die tatsächlichen Labels als neue Spalten zum DataFrame hinzu
df['predicted_sentiment'] = predicted_sentiments

# Vergleichen Sie die vorhergesagten Sentiments mit den tatsächlichen Labels
correct_predictions = (df['predicted_sentiment'] == df['label']).sum()
total_predictions = len(df)

# Berechnen Sie die Genauigkeit (Accuracy) der Vorhersagen
accuracy = correct_predictions / total_predictions

# Drucken Sie die Genauigkeit
print(f"Genauigkeit: {accuracy:.2%}")


Genauigkeit: 54.80%


In [10]:
df.head()

,text,label,predicted_sentiment
0,Dark Souls 3 April Launch Date Confirmed With ...,1,1
1,"""National hot dog day, national tequila day, t...",2,1
2,When girls become bandwagon fans of the Packer...,0,1
3,@user I may or may not have searched it up on ...,1,1
4,Here's your starting TUESDAY MORNING Line up a...,1,1


In [11]:
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Ihre Verwirrungsmatrix
conf_matrix = confusion_matrix(df['label'], df['predicted_sentiment'])

# Berechnen Sie die Precision und den Recall für jede Klasse
precision = precision_score(df['label'], df['predicted_sentiment'], average=None)
recall = recall_score(df['label'], df['predicted_sentiment'], average=None)

# Berechnen Sie den gewichteten Durchschnitt von Precision und Recall (Macro-Durchschnitt)
macro_precision = precision_score(df['label'], df['predicted_sentiment'], average='macro')
macro_recall = recall_score(df['label'], df['predicted_sentiment'], average='macro')

# Erstellen Sie ein DataFrame für die Verwirrungsmatrix
confusion_df = pd.DataFrame(conf_matrix, columns=["Predicted Class 0", "Predicted Class 1", "Predicted Class 2"], index=["True Class 0", "True Class 1", "True Class 2"])

# Drucken Sie die Verwirrungsmatrix
print("Verwirrungsmatrix:")
print(confusion_df)

# Erstellen Sie ein DataFrame für Precision und Recall
precision_recall_df = pd.DataFrame({"Precision": precision, "Recall": recall}, index=["Class 0", "Class 1", "Class 2"])

# Drucken Sie die Precision und den Recall für jede Klasse
print("\nPrecision und Recall pro Klasse:")
print(precision_recall_df)

# Drucken Sie den gewichteten Durchschnitt von Precision und Recall (Macro-Durchschnitt)
print(f"\nMacro Precision: {macro_precision:.2f}")
print(f"Macro Recall: {macro_recall:.2f}")


Verwirrungsmatrix:
              Predicted Class 0  Predicted Class 1  Predicted Class 2
True Class 0                 65                210                 37
True Class 1                 57                672                140
True Class 2                 24                436                359

Precision und Recall pro Klasse:
         Precision    Recall
Class 0   0.445205  0.208333
Class 1   0.509863  0.773303
Class 2   0.669776  0.438339

Macro Precision: 0.54
Macro Recall: 0.47
